In [ ]:
import numpy as np
import pandas as pd
from qiskit import IBMQ
from qiskit import QuantumCircuit, execute, Aer, QuantumRegister, transpile, assemble
from qiskit.tools.monitor import job_monitor
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)
from qiskit.visualization import plot_histogram
import json
import pickle
import math

#np.random.seed(2022321127)

IBMQ.save_account("... ", overwrite=True)
IBMQ.load_account() # Load account from disk
provider = IBMQ.get_provider(hub='ibm-q-skku', group='yonsei-uni', project='yu-graduate')



In [ ]:
class data_collection:

    def __init__(self, n_data = 100, times = 1, n_qubits = 5, shots = 1024, device_type = "simulator_statevector", LI_QREM = True, qubits = [0, 1, 2, 3, 4], t_qubit = 27, number = 0):
        self.n_data = n_data # number of data (circuit)
        self.times = times
        self.n_qubits = n_qubits # number of qubits
        self.SHOTS = shots
        self.device_type = device_type
        self.device = provider.get_backend(device_type)
        self.LI_QREM = LI_QREM 
        
        self.qubits = qubits # qubits number
        self.t_qubit = t_qubit # total qubits
        self.number = number

        self.theta = []
        # generate theta
        for i in range(n_data * times):
            x = np.random.uniform(-1, 1, size = n_qubits)
            y = []
            for j in x:
                y.append(math.acos(j))

            y = np.array(y)
            self.theta.append(y)
        
        self.get_ideal_prob()
        self.get_obs_prob()

    def get_ideal_prob(self):
        if self.LI_QREM:
            file_name = self.device_type + '_' + str(self.n_qubits) + 'qubits_ideal_' + str(self.number) + '.json'
        else:
            file_name = self.device_type + '_' + str(self.n_qubits) + 'qubits_ideal_no_LI' + '.json'
        
        for t in range(self.times):
            x = []
            for k in range(self.n_data):
                prob = {}
                for i in range(2**self.n_qubits):
                    bit_i = str(bin(i)[2:]).zfill(self.n_qubits)
                    p = 1
                    for j in range(self.n_qubits):
                        if bit_i[self.n_qubits - 1 - j] == '0':
                            p *= np.cos(self.theta[k + (t * self.n_data)][j] / 2)
                        else:
                            p *= np.sin(self.theta[k + (t * self.n_data)][j] / 2)

                    prob[bit_i] = p ** 2

                x.append(prob)
            self.save_data(x, file_name)
            

    def get_obs_prob(self):
        if self.LI_QREM:
            #qr = QuantumRegister(self.n_qubits)
            meas_calibs, state_labels = complete_meas_cal(self.qubits, circlabel = 'mcal')
            cal_results = execute(meas_calibs, self.device, shots = self.SHOTS).result()
            meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel = 'mcal')
            meas_filter = meas_fitter.filter
            
            f = self.device_type + '_' + str(self.n_qubits) + 'qubits_meas_' + str(self.number) + '.p'
            with open(f, 'wb') as file:  
                pickle.dump(meas_fitter, file)

        if self.LI_QREM:
            file_name = self.device_type + '_' + str(self.n_qubits) + 'qubits_obs_' + str(self.number) + '.json'
        else:
            file_name = self.device_type + '_' + str(self.n_qubits) + 'qubits_obs_no_LI' + '.json'
            
        for t in range(self.times):
            total_qc = []
            for x in range(self.n_data):
                qc = QuantumCircuit(self.t_qubit, self.n_qubits)
                for i in range(self.n_qubits):
                    qc.ry(self.theta[x + (t * self.n_data) ][i], self.qubits[i])
                    qc.measure(self.qubits[i], i)
                total_qc.append(qc)

            real_data = execute(total_qc, self.device, shots = self.SHOTS).result().get_counts()

            for r_n in range(len(real_data)):
                for i in range(2**self.n_qubits):
                    bit_i = str(bin(i)[2:]).zfill(self.n_qubits)
                    if bit_i not in real_data[r_n]:
                        real_data[r_n][bit_i] = 0.0
                    else:
                        real_data[r_n][bit_i] /= self.SHOTS

            self.save_data(real_data, file_name)

    def save_data(self, data, file_path):
        json_data = []
        try :
            with open(file_path, "r") as json_file:
                json_data = json.load(json_file)

            json_data.append(data)

            with open(file_path, 'w') as f:
                json.dump(json_data, f)

        except FileNotFoundError as e :
            json_data = []
            json_data.append(data)
            with open(file_path, 'w') as f:
                json.dump(json_data, f)



In [ ]:
# Ex) generate 5qubit 9000data 

n_data = 300
times = 30
n_qubit = 5
shot = 8192
device_type = "ibm_geneva"
LI_QREM = True
qubits = [0, 1, 2, 3, 4]
t_qubit = 27

data_collection(n_data, times, n_qubit, shot, device_type, LI_QREM, qubits, t_qubit, 0)